<a href="https://colab.research.google.com/github/xg6144/Machine_Practice/blob/main/Tree_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#트리검증
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state=42
)

sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state = 42
)
#훈련세트 sub_input, sub_target 검증 세트 val_input, val_target
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [5]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.8653846153846154


In [6]:
#교차 검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00718379, 0.00659394, 0.00704575, 0.00709987, 0.00667858]), 'score_time': array([0.00072598, 0.00063729, 0.00068355, 0.00063419, 0.00063205]), 'test_score': array([0.86826923, 0.85096154, 0.87102984, 0.85274302, 0.84119346])}


In [7]:
import numpy as np
print(np.mean(scores['test_score']))

0.8568394165988007


In [9]:
from sklearn.model_selection import StratifiedKFold #훈련세트를 섞는다.

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) #cross_validate()는 기본적으로 StratifiedKFold()를 사용한다.
print(np.mean(scores['test_score']))

0.8562623084326646


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) #10폴드 교차 검증
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8587653772046835


In [14]:
#Hyper Parameter Tuning
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} #불순도 감소 최소량

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=1)

gs.fit(train_input, train_target)
dt = gs.best_estimator_ #최적의 모델을 저장
print(dt.score(train_input, train_target))
print(gs.best_params_)#최적의 매개변수

0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [15]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [16]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [26]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
          'max_depth':range(5,20,1),
          'min_samples_split':range(2,100,10) #노드를 나누기 위한 최소 샘플 수 
          }

gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [27]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [28]:
from scipy.stats import uniform, randint

rgen = randint(0,10)
rgen.rvs(10)

array([3, 5, 3, 8, 1, 1, 8, 3, 6, 4])

In [29]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 95, 109,  94, 108,  95,  88, 110, 112, 108,  81]))

In [30]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.33795427, 0.57417947, 0.9730153 , 0.03099713, 0.44807773,
       0.89009695, 0.24500577, 0.41366892, 0.38421048, 0.83151798])

In [31]:
params = {'min_impurity_decrease':uniform(0.0001, 0.001),
          'max_depth':randint(20,50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25),
          }

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [35]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [36]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


In [37]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.786923076923077
